# Variational classifier

The notebook has two purposes, tackled in two examples. In both, a variational circuit is trained. In the first one, the aim is to learn the parity function, which requires that a binary input is encoded. In the second example, the Iris dataset is learned, which requires real amplitude vectors to be encoded as amplitudes.

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

# Fitting the parity function

In [3]:
n_qubits = 4

In [4]:
dev = qml.device("default.qubit", wires=n_qubits)

### Functions to prepare the circuit

In [25]:
def layer(W):
    """Applies a layer of arbitrary rotations and circular entanglements to the variational circuit

    Args:
        W (np.ndarray): rotation parameters for the layer
    """
    for i in range(n_qubits):
        qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)

    for i in range(n_qubits-1):
        qml.CNOT(wires=[i, i+1])

    if n_qubits > 2:
        qml.CNOT(wires=[n_qubits-1, 0])

In [6]:
def statepreparation(x):
    """Prepares the binary state fed to the vqc

    Args:
        x (List): list of 0s and 1s corresponding to the basis state
    """
    qml.BasisState(x, wires=[i for i in range(n_qubits)])

### Defining the `qnode`

In [19]:
@qml.qnode(dev)
def circuit(weights, x):
    """Generates the trainable quantum circuit with binary input

    Args:
        weights (List[np.ndarray]): all the variational parameters, one
            array per layer
        x (List): binaries representing the input
    """
    statepreparation(x)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

### Adding a classical bias parameter

In [20]:
def variational_classifier(weights, bias, x):
    """Complete variational classifier, including a bias

    Args:
        weights (List[np.ndarray]): all the variational parameters, one
            array per layer
        bias (float): classical bias value, added to the output of the quantum circuit
        x (List): binaries representing the input
    """
    return circuit(weights, x) + bias

## Cost function

In [21]:
def square_loss(labels, predictions):
    """Computes the MSE between labels and predictions

    Args:
        labels (List[int]): actual values
        predictions (List[int]): model predictions

    Returns:
        float: value of the MSE
    """
    loss = 0.
    for l, p in zip(labels, predictions):
        loss += (l - p) ** 2

    loss /= len(labels)
    return loss

In [22]:
def accuracy(labels, predictions):
    """Returns the accuracy over the dataset

    Args:
        labels (List[int]): true values
        predictions (List[int]): model-predicted values

    Returns:
        float: number of accurate predictions over total
    """
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss += 1

    loss /= len(labels)
    return loss

In [23]:
def cost(weights, bias, X, Y):
    """Computes the predictions and returns the MSE over the dataset

    Args:
        weights (List[np.ndarray]): all the variational parameters, one
            array per layer
        bias (float): classical bias value, added to the output of the quantum circuit
        X (List[List]): list of all the binary input strings
        Y (List): labels

    Returns:
        float: MSE over the dataset
    """
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

## Optimization

In [7]:
data = np.loadtxt("variational_classifier/data/parity.txt")

In [8]:
X = np.array(data[:, :-1], requires_grad=False)
Y = np.array(data[:, -1], requires_grad=False)

# shift lables from [0, 1] to [-1, 1], to match the range of expectation values
Y = 2 * Y - np.ones(len(Y))

### Initialization

In [16]:
rng = np.random.default_rng(0)

num_layers = 2
weights_init = .01 * rng.normal(size=(num_layers, n_qubits, 3), requires_grad=True)
bias_init = np.array(0., requires_grad=True)

print(weights_init, bias_init)

[[[ 0.0012573  -0.00132105  0.00640423]
  [ 0.001049   -0.00535669  0.00361595]
  [ 0.01304     0.00947081 -0.00703735]
  [-0.01265421 -0.00623274  0.00041326]]

 [[-0.02325031 -0.00218792 -0.01245911]
  [-0.00732267 -0.00544259 -0.003163  ]
  [ 0.00411631  0.01042513 -0.00128535]
  [ 0.01366463 -0.00665195  0.0035151 ]]] 0.0


### Defining optimizer and batch size

In [17]:
opt = NesterovMomentumOptimizer(.5)
batch_size = 5

### Optimize

In [26]:
weights = weights_init
bias = bias_init
iterations = 25

for it in range(iterations):

    # shuffle the batch indices
    batch_index = rng.integers(0, len(X), size=batch_size)

    X_batch = X[batch_index]
    Y_batch = Y[batch_index]

    # update weights and biases
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

Iter:     1 | Cost: 2.1594871 | Accuracy: 0.5000000 
Iter:     2 | Cost: 2.1571387 | Accuracy: 0.5000000 
Iter:     3 | Cost: 3.3674659 | Accuracy: 0.5000000 
Iter:     4 | Cost: 1.3990385 | Accuracy: 0.5000000 
Iter:     5 | Cost: 1.0404263 | Accuracy: 0.5000000 
Iter:     6 | Cost: 0.9132724 | Accuracy: 0.5000000 
Iter:     7 | Cost: 0.1498629 | Accuracy: 1.0000000 
Iter:     8 | Cost: 0.0293096 | Accuracy: 1.0000000 
Iter:     9 | Cost: 0.2336926 | Accuracy: 1.0000000 
Iter:    10 | Cost: 0.3322166 | Accuracy: 1.0000000 
Iter:    11 | Cost: 0.2453625 | Accuracy: 1.0000000 
Iter:    12 | Cost: 0.0481789 | Accuracy: 1.0000000 
Iter:    13 | Cost: 0.0038142 | Accuracy: 1.0000000 
Iter:    14 | Cost: 0.0038158 | Accuracy: 1.0000000 
Iter:    15 | Cost: 0.0334435 | Accuracy: 1.0000000 
Iter:    16 | Cost: 0.0920158 | Accuracy: 1.0000000 
Iter:    17 | Cost: 0.0215532 | Accuracy: 1.0000000 
Iter:    18 | Cost: 0.0138518 | Accuracy: 1.0000000 
Iter:    19 | Cost: 0.0060476 | Accuracy: 1.00